In [1]:
# Turn americangut into a binary classification problem


In [2]:
import anndata


In [3]:
adata = anndata.read_h5ad("data/americangut_embeddings.h5ad")


In [11]:
def report(column, normalize=False):
    """Report on column values. Call this for manual inspection"""
    print(adata.obs[column].str.lower().value_counts(normalize=normalize))


report("ibd")


ibd
i do not have this condition                                         26122
nan                                                                   5137
diagnosed by a medical professional (doctor, physician assistant)     1089
self-diagnosed                                                         219
diagnosed by an alternative medicine practitioner                       37
not provided                                                             3
unspecified                                                              1
Name: count, dtype: int64


In [10]:
def classify(column, pos_vals, neg_vals, nan_vals, data=adata.X):
    vals = adata.obs[column].str.lower()
    nan_vals = vals.isna() | vals.isin(nan_vals)
    pos_vals = vals.isin(pos_vals)
    neg_vals = vals.isin(neg_vals)
    assert not (pos_vals & neg_vals).any()
    assert (pos_vals | neg_vals | nan_vals).all()

    X = data[~nan_vals]
    y = pos_vals[~nan_vals]

    return X, y


In [12]:
X, y = classify(
    column="ibd",
    pos_vals=[
        "diagnosed by a medical professional (doctor, physician assistant)",
        "self-diagnosed",
        "diagnosed by an alternative medicine practitioner",
    ],
    neg_vals=["i do not have this condition"],
    nan_vals=["nan", "not provided", "unspecified"],
    data=adata.obsm["euc_mix_128"],
)


In [27]:
# Use this to build up a master list

for col in adata.obs.columns[50:60]:
    print(adata.obs.dtypes[col])
    if adata.obs.dtypes[col] == "category":
        report(col)


category
cancer_treatment
nan                  31048
surgery only           847
chemotherapy           260
not provided           188
radiation therapy      173
no treatment            87
unspecified              5
Name: count, dtype: int64
category
cardiovascular_disease
i do not have this condition                                         22371
nan                                                                   9158
diagnosed by a medical professional (doctor, physician assistant)     1041
self-diagnosed                                                          30
diagnosed by an alternative medicine practitioner                        6
not provided                                                             2
Name: count, dtype: int64
category
cat
no              21186
yes              7243
nan              4171
false               6
not provided        1
true                1
Name: count, dtype: int64
category
cdiff
i do not have this condition                                     

In [22]:
neg_master = ["no", "false", "i do not have this condition", "never"]

pos_master = [
    "yes",
    "true",
    "diagnosed by a medical professional (doctor, physician assistant)",
    "self-diagnosed",
    "diagnosed by an alternative medicine practitioner",
    "rarely (less than once/week)",
    "occasionally (1-2 times/week)",
    "regularly (3-5 times/week)",
    "daily",
]

nan_master = ["nan", "not provided", "not sure", "not collected"]
